In [ ]:
from openai import OpenAI
client = OpenAI(api_key="#####")

In [2]:
import pandas as pd
import json

In [3]:
items = pd.read_csv("./kaggle_data/items.csv")
items['Title'] = items['Title'].str.replace(r'\s*/\s*$', '', regex=True)
items

,Title,Author,ISBN Valid,Publisher,Subjects,i
0,Classification décimale universelle : édition ...,NaN,9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1
2,Histoire de vie et recherche biographique : pe...,NaN,2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3
4,Les années glorieuses : roman,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4
...,...,...,...,...,...,...
15286,Le vagabond de Tokyo,"Fukutani, Takashi, 1952-2000",9782353480111; 235348011X; 9782353480241; 2353...,Le Lézard noir,Mangas,15286
15287,God of high school : le match contre les dieux,"Park, Yong-Je",9782382880203; 2382880201; 9782382880210; 2382...,Kbooks,NaN,15287
15288,Blue Lock,"Kaneshiro, Muneyuki",9782811650254; 2811650253; 9782811661274; 2811...,Pika,Compétitions; Football; Entraînement (sports);...,15288
15289,Red eyes sword : akame ga kill ! Zero,Takahiro,9782368522134; 2368522131; 9782368522141; 2368...,Kurokawa,Bandes dessinées; Mangas,15289


In [ ]:
system_message = """
You are a professional book metadata assistant.

Given a JSON with:
• "title": French book title
• "subjects": list of French keywords

Return only a JSON with:

"Semantic_Tags": up to 3 concise English nouns/phrases
"Genre": 1 top-level English genre
"Subgenre": 1 specific English subgenre
"Themes": up to 2 high-level English themes
"Tone_Mood": up to 3 English descriptors (tone or style)
"Target_Audience": English phrase, ≤10 words
"Short_Summary": 1–2 English sentences, max 20 words
Output only valid JSON. No explanations or extra text.
"""

In [6]:
tasks = []

for index, row in items[10000:].iterrows():

    title = row["Title"]
    subjects = row["Subjects"]
    input_dict = {
        "title": title,
        "subjects": subjects
    }
    
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4.1-mini",
            "temperature": 0.1,
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": system_message
                },
                {
                    "role": "user",
                    "content": json.dumps(input_dict, ensure_ascii=False)
                }
            ],
        }
    }
    
    tasks.append(task)

In [7]:
file_name = "./batch_tasks_books_2.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj, ensure_ascii=False) + '\n')

In [8]:
batch_file = client.files.create(file=open(file_name, "rb"), purpose="batch")

In [9]:
print(batch_file)

FileObject(id='file-SDJu6P12z6aXvuqj381MWR', bytes=5415573, created_at=1747298636, filename='batch_tasks_books_2.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)


In [10]:
batch_job = client.batches.create(
    input_file_id=batch_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

In [11]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_68252ded32fc8190a668f0c0b9ed5ef1', completion_window='24h', created_at=1747267053, endpoint='/v1/chat/completions', input_file_id='file-4AVWr9c2f3iMAsero3VnDL', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1747353453, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
